In [409]:
print "ready"

ready


In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime
import time
import matplotlib.pyplot as plt
import os
import re
from dateutil.relativedelta import relativedelta

In [5]:
path = 'E:/SUBJECTS/497 MATH S17 PSP--/Data/'
interpath= 'E:/SUBJECTS/497 MATH S17 PSP--/Data/intermediate/'

In [4]:
encounters=pd.read_pickle(path + '/encounters.pickle')

In [18]:
print encounters.shape
encounters.info()

(85829, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 85829 entries, 10335 to 47927
Data columns (total 5 columns):
﻿Person_ID       85829 non-null object
Person_Nbr       85829 non-null int64
Enc_ID           85829 non-null object
Enc_Nbr          85829 non-null int64
Enc_Timestamp    85829 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 3.9+ MB


There are 85829 unique Enc_Nbrs and 16509 unique Person_Nbrs in encounters table

In [25]:
print len(encounters.Enc_Nbr.unique())
len(encounters.Person_Nbr.unique())

85829


16509

In [17]:
all_enc = pd.read_pickle(path + '/all_encounter_data.pickle') 

In [112]:
print all_enc.shape
all_enc.info()

(82432, 41)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82432 entries, 75857 to 79638
Data columns (total 41 columns):
Enc_ID            82432 non-null object
Enc_Nbr           82432 non-null int64
Enc_Date          82432 non-null datetime64[ns]
Person_ID         82432 non-null object
Person_Nbr        82432 non-null int64
Primary_Payer     82432 non-null object
Smoking_Status    59199 non-null object
BMI               22325 non-null object
BP                38219 non-null object
Glucose           30483 non-null object
A1C               12793 non-null float64
MR_OD_SPH         25476 non-null object
MR_OD_CYL         24693 non-null object
MR_OD_AXIS        19897 non-null object
MR_OD_DVA         24179 non-null object
MR_OD_NVA         18104 non-null object
MR_OS_SPH         25452 non-null object
MR_OS_CYL         24606 non-null object
MR_OS_AXIS        19417 non-null object
MR_OS_DVA         24068 non-null object
MR_OS_NVA         18035 non-null object
BB_OD_SPH         5851 non-n

There are 80946 unique Enc_Nbrs and 16087 unique Person_Nbrs in all_enc table

In [27]:
print len(all_enc.Enc_Nbr.unique())
uni

80946


16087

In [458]:
print all_enc.Enc_Date.min()
all_enc.Enc_Date.max()

2011-10-28 23:00:00


Timestamp('2016-11-03 23:15:00')

## Findings: 

There are 3397 Enc_nbrs in encounter table which do not have data recorded for them in the all_enc table
There are 422 Person_nbrs in encounter table which do not have data recorded for them in the all_enc table

Out of the 82432 Enc_Nbrs in the all_enc table 80946 are unique leaving 1486 duplicated Enc_Nbrs

In [28]:
len(all_enc['Enc_Nbr'])-len(all_enc['Enc_Nbr'].drop_duplicates())

1486

## IDEA: Create a table for each person with a single encounter data set where examination values are averaged

## Distribution and Outliers  

Separate BP into BP_Systolic and BP_Diastolic

In [73]:
pattern = re.compile("(?P<BP_Systolic>\d+)\s*\/\s*(?P<BP_Diastolic>\d+)")
all_enc = pd.merge(all_enc, all_enc["BP"].str.extract(pattern, expand=True),
                         left_index=True, right_index=True)

Fix the 87 values in Glucose column with x/y format

In [75]:
result=all_enc["Glucose"].str.extract(pattern, expand=True)
result.info()

In [77]:
all_enc=pd.merge(all_enc,result,left_index=True,right_index=True)
all_enc.head()

,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,Primary_Payer,Smoking_Status,BMI,BP,Glucose,...,CYCLO_OD_NVA,CYCLO_OS_SPH,CYCLO_OS_CYL,CYCLO_OS_AXIS,CYCLO_OS_DVA,CYCLO_OS_NVA,BP_Systolic_x,BP_Diastolic_x,BP_Systolic_y,BP_Diastolic_y
75857,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,Premier Vision Plan,Never smoker,NaN,NaN,130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75858,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,Premier Vision Plan,Never smoker,NaN,118/ 78,130,...,NaN,NaN,NaN,NaN,NaN,NaN,118,78,NaN,NaN
55347,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,Adv BCBS HMO,Former smoker,26.04,140 / 88,140,...,NaN,NaN,NaN,NaN,NaN,NaN,140,88,NaN,NaN
55348,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,Adv BCBS HMO,Former smoker,29.16,128 / 78,120,...,NaN,NaN,NaN,NaN,NaN,NaN,128,78,NaN,NaN
55349,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,Adv BCBS HMO,Former smoker,29.16,132 / 86,130,...,NaN,NaN,NaN,NaN,NaN,NaN,132,86,NaN,NaN


In [79]:
print all_enc.BP_Systolic_y.isnull().value_counts()
all_enc.BP_Diastolic_y.isnull().value_counts()

True     82345
False       87
Name: BP_Systolic_y, dtype: int64


True     82345
False       87
Name: BP_Diastolic_y, dtype: int64

In [80]:
all_enc.BP_Systolic_x=all_enc.BP_Systolic_x.convert_objects(convert_numeric=True)
all_enc.BP_Systolic_y=all_enc.BP_Systolic_y.convert_objects(convert_numeric=True)
BPS = pd.concat((all_enc.BP_Systolic_x, all_enc.BP_Systolic_y),axis=1).mean(axis=1)
all_enc.BP_Systolic_x=BPS

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [81]:
all_enc.BP_Diastolic_x=all_enc.BP_Diastolic_x.convert_objects(convert_numeric=True)
all_enc.BP_Diastolic_y=all_enc.BP_Diastolic_y.convert_objects(convert_numeric=True)
BPD = pd.concat((all_enc.BP_Diastolic_x, all_enc.BP_Diastolic_y),axis=1).mean(axis=1)
all_enc.BP_Diastolic_x=BPD

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [82]:
pattern1 = re.compile("(\d+)\s*\/\s*(\d+)")
all_enc.Glucose=all_enc.Glucose.replace(pattern1,"NaN")

Converting to int 

In [84]:
toconvert=['A1C','BMI','Glucose']
for i in toconvert:
    all_enc[i]=all_enc[i].convert_objects(convert_numeric=True)

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


Change NaN to zero

In [86]:
tochange=['A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']
for i in tochange:
    all_enc[i]=np.nan_to_num(all_enc[i])

Create a new set for analysis 

In [88]:
columns=['Person_Nbr','Enc_Nbr','Enc_Date','Smoking_Status','Primary_Payer','A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']

In [92]:
all_enc_1= pd.DataFrame(columns=columns)

In [94]:
for index,row in all_enc.iterrows():
    for i in columns:
           all_enc_1.loc[index,i]=row[i]

In [99]:
all_enc_1.head()

,Person_Nbr,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x
75857,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8,0,130,0,0
75858,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8,0,130,118,78
55347,89,12870648,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0,26.04,140,140,88
55348,89,4126172,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0,29.16,120,128,78
55349,89,9217142,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0,29.16,130,132,86


In [102]:
all_enc_1.to_csv('all_enc_1.txt')

In [101]:
all_enc_1.shape

(82432, 10)

Based on the visualizations in R and research the following outlier ranges have been considered 

In [11]:
all_enc_2=pd.read_csv(interpath+'all_enc_1.csv')

In [13]:
all_enc_2.head()

,Unnamed: 0,Person_Nbr,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x
0,75857,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,0.0,0.0
1,75858,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,118.0,78.0
2,55347,89,12870648,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0
3,55348,89,4126172,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0
4,55349,89,9217142,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0


In [15]:
outliers = {
            "A1C" : lambda x: x if 3.89 < x < 18 else 0,
            "BMI" : lambda x: x if 8 < x < 204 else 0,
            "Glucose" : lambda x: x if 50 < x < 2656 else 0,
            "BP_Systolic_x" : lambda x: x if 55 < x < 370 else 0,
            "BP_Diastolic_x" : lambda x: x if 30 < x < 360 else 0
        }
       

In [16]:
 for column in list(outliers):
            all_enc_2[column] = all_enc_2[column].map(outliers[column])

In [17]:
all_enc_2.to_csv('all_enc_2.csv')

Taking average for the values recorded for same encounter, i.e the 1486 duplicated Enc_Nbrs

In [50]:
all_enc_2=pd.read_csv(interpath+'all_enc_2.csv')

In [51]:
dups= pd.DataFrame(columns=dict(all_enc_2))

In [52]:
all_enc_2["Duplicate_Exists"]=all_enc_2["Enc_Nbr"].duplicated(keep=False)

In [53]:
all_enc_2.Duplicate_Exists.sum()

2806

In [54]:
for index,row in all_enc_2.iterrows():
    if row["Duplicate_Exists"]==True:
        dups.loc[index]=row

In [55]:
dups.shape

(2806, 12)

In [58]:
dups.head()

,BP_Diastolic_x,BMI,Smoking_Status,BP_Systolic_x,Enc_Date,A1C,Enc_Nbr,Primary_Payer,Person_Nbr,Glucose
0,0.0,0.0,Never smoker,0.0,2014-12-18 14:00:00,8.0,123227.0,Premier Vision Plan,33.0,130.0
1,78.0,0.0,Never smoker,118.0,2014-12-18 14:00:00,8.0,123227.0,Premier Vision Plan,33.0,130.0
34,106.0,0.0,Never smoker,180.0,2015-03-03 18:15:00,0.0,13886332.0,Blue Shield Of Illinois PPO,535.0,0.0
35,55.0,0.0,Never smoker,124.0,2015-03-03 18:15:00,0.0,13886332.0,Blue Shield Of Illinois PPO,535.0,0.0
36,55.0,24.6,Never smoker,124.0,2015-03-03 18:15:00,0.0,13886332.0,Blue Shield Of Illinois PPO,535.0,0.0


In [57]:
del dups["Unnamed: 0"]
del dups["Unnamed: 0.1"]

In [59]:
dups.to_csv('dups.csv')

In [61]:
columns=['A1C','BMI','Glucose','BP_Systolic_x','BP_Diastolic_x']

In [62]:
columns1=['Enc_Nbr','Enc_Date','Person_Nbr']

In [63]:
columns2=['Primary_Payer','Smoking_Status']

Average for numeric values

In [64]:
a=dups.groupby("Enc_Nbr")[columns].mean()

In [65]:
a.shape

(1320, 5)

In [66]:
a.head()

,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x
Enc_Nbr,,,,,
10196.0,0.0,0.00,0.0,126.5,65.0
31148.0,0.0,31.87,0.0,142.0,82.0
44311.0,7.0,0.00,105.0,70.0,45.0
46178.0,0.0,0.00,282.0,153.5,95.0
69786.0,0.0,0.00,131.0,189.5,99.0


Check change in Smoking status and PP

In [298]:
index=0
b=pd.DataFrame(columns=['Enc_Nbr','Person_Nbr',"Primary_Payer","Num_PP", 'Smoking_Status',"Num_SS"])
for name, group in dups.groupby(['Enc_Nbr']):
    b.loc[index,"Enc_Nbr"]=name
    b.loc[index,"Person_Nbr"]=group.get_value(index=0,col=8,takeable=True)
    b.loc[index,"Smoking_Status"]=group.get_value(index=0,col=2,takeable=True)
    b.loc[index,"Num_SS"]=1
    b.loc[index,"Primary_Payer"]=group.get_value(index=0,col=7,takeable=True)
    b.loc[index,"Num_PP"]=1
    for i in range(len(group.get_values())-1):
        frsts= b.loc[index,"Smoking_Status"]
        secs= group.get_value(index=i+1,col=2,takeable=True)
        if frsts!=secs and pd.isnull(secs)==False:
            boths=[frsts,secs]
            b.loc[index,"Smoking_Status"]=boths
            b.loc[index,"Num_SS"]=len(boths)
        frstp= b.loc[index,"Primary_Payer"]
        secp= group.get_value(index=i+1,col=7,takeable=True)
        if frstp!=secp and pd.isnull(secp)==False:
            bothp=[frstp,secp]
            b.loc[index,"Primary_Payer"]=bothp
            b.loc[index,"Num_PP"]=len(bothp)       
    index=index+1     

In [299]:
print b.shape
b=b.set_index("Enc_Nbr")
b.head()

(1320, 6)


,Person_Nbr,Primary_Payer,Num_PP,Smoking_Status,Num_SS
Enc_Nbr,,,,,
10196.0,590786,Medicare,1,Former smoker,1
31148.0,203064,VOHHA,1,Former smoker,1
44311.0,329550,Medicaid HFS,1,Never smoker,1
46178.0,735932,Medicare,1,Current some day smoker,1
69786.0,265609,Medicaid HFS,1,Never smoker,1


In [300]:
print sum(b["Num_PP"]>1)
sum(b["Num_SS"]>1)

0


0

In [209]:
b=dups.drop(columns, axis=1).drop_duplicates("Enc_Nbr").set_index("Enc_Nbr")

In [210]:
b.shape

(1320, 5)

In [211]:
b.head()

,Unnamed: 0,Smoking_Status,Enc_Date,Primary_Payer,Person_Nbr
Enc_Nbr,,,,,
123227.0,0,Never smoker,2014-12-18 14:00:00,Premier Vision Plan,33.0
13886332.0,2,Never smoker,2015-03-03 18:15:00,Blue Shield Of Illinois PPO,535.0
5330366.0,5,Never smoker,2016-06-10 18:30:00,Medicare,777.0
4612609.0,7,Former smoker,2012-01-12 00:00:00,Medicare,1614.0
2586704.0,11,Never smoker,2012-05-01 06:00:00,Medicare,2556.0


In [215]:
del b["Unnamed: 0"]

In [45]:
b.head()

NameError: name 'b' is not defined

In [218]:
dups1=pd.merge(a,b,left_index=True,right_index=True)

In [249]:
dups

(1320, 9)

In [220]:
dups1.head()

,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Smoking_Status,Enc_Date,Primary_Payer,Person_Nbr
Enc_Nbr,,,,,,,,,
123227.0,8.0,0.000,130.0,59.000000,39.0,Never smoker,2014-12-18 14:00:00,Premier Vision Plan,33.0
13886332.0,0.0,8.200,0.0,142.666667,72.0,Never smoker,2015-03-03 18:15:00,Blue Shield Of Illinois PPO,535.0
5330366.0,0.0,0.000,188.0,163.000000,95.0,Never smoker,2016-06-10 18:30:00,Medicare,777.0
4612609.0,3.3,14.445,36.0,55.000000,33.0,Former smoker,2012-01-12 00:00:00,Medicare,1614.0
2586704.0,0.0,19.080,0.0,0.000000,0.0,Never smoker,2012-05-01 06:00:00,Medicare,2556.0


In [222]:
dups1.to_csv('dups1.csv')

Merge with full data

In [223]:
all_enc_2.shape

(82432, 12)

In [224]:
all_enc_2.head()

,Unnamed: 0,Person_Nbr,Enc_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Duplicate_Exists
0,75857,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,0.0,0.0,True
1,75858,33,123227,2014-12-18 14:00:00,Never smoker,Premier Vision Plan,8.0,0.00,130.0,118.0,78.0,True
2,55347,89,12870648,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,False
3,55348,89,4126172,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,False
4,55349,89,9217142,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,False


In [250]:
c=all_enc_2.drop_duplicates("Enc_Nbr",keep=False).set_index("Enc_Nbr")

In [251]:
c.shape

(79626, 11)

In [253]:
del c["Unnamed: 0"]

In [254]:
c.head()

,Person_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Duplicate_Exists
Enc_Nbr,,,,,,,,,,
12870648,89,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,False
4126172,89,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,False
9217142,89,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,False
14968450,89,2014-05-25 10:45:00,Former smoker,Adv BCBS HMO,0.0,26.38,130.0,78.0,62.0,False
14537805,89,2015-03-14 15:30:00,Former smoker,Adv BCBS HMO,0.0,0.00,0.0,125.0,73.0,False


In [230]:
for i,j in c.iterrows():
    if j["Duplicate_Exists"]==True:
        print j    

In [256]:
c=c.reset_index()
dups1=dups1.reset_index()

In [257]:
all_enc_3=pd.merge(c,dups1,how="outer")

In [258]:
all_enc_3.shape

(80946, 12)

In [259]:
all_enc_3.head()

,index,Enc_Nbr,Person_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,Duplicate_Exists
0,0.0,12870648.0,89.0,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,False
1,1.0,4126172.0,89.0,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,False
2,2.0,9217142.0,89.0,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,False
3,3.0,14968450.0,89.0,2014-05-25 10:45:00,Former smoker,Adv BCBS HMO,0.0,26.38,130.0,78.0,62.0,False
4,4.0,14537805.0,89.0,2015-03-14 15:30:00,Former smoker,Adv BCBS HMO,0.0,0.00,0.0,125.0,73.0,False


In [260]:
all_enc_3.to_csv('all_enc_3.csv')

Adding ICD codes

In [262]:
icd_for_enc=pd.read_pickle(path+'ICD_for_Enc.pickle')

In [263]:
print icd_for_enc.shape
icd_for_enc.info()

(207255, 7)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 207255 entries, 194529 to 201888
Data columns (total 7 columns):
Person_ID            207255 non-null object
Person_Nbr           207255 non-null int64
Enc_ID               207255 non-null object
Enc_Nbr              207255 non-null int64
Enc_Timestamp        207255 non-null datetime64[ns]
Diagnosis_Code_ID    207255 non-null object
Description          207251 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 12.6+ MB


In [264]:
icd_for_enc.head()

,Person_ID,Person_Nbr,Enc_ID,Enc_Nbr,Enc_Timestamp,Diagnosis_Code_ID,Description
194529,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,367.4,Presbyopia
194530,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,373.00,Blepharitis unspecified
194531,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,250.00,Diabetes mellitus without mention of complicat...
136763,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,365.02,Anatomical narrow angle borderline glaucoma
136764,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,250.00,Diabetes Mellitus Type 2 Uncomplicated


In [265]:
len(icd_for_enc.Enc_Nbr.unique())

64217

In [266]:
icd_for_enc.Diagnosis_Code_ID.describe()

count     207255
unique      2139
top       250.00
freq       18575
Name: Diagnosis_Code_ID, dtype: object

In [267]:
cols=["Enc_Nbr","Person_Nbr","Diagnosis_Code_ID"]

In [268]:
icd=icd_for_enc[cols]

In [269]:
icd=icd.reset_index()

In [270]:
icd.head()

,index,Enc_Nbr,Person_Nbr,Diagnosis_Code_ID
0,194529,123227,33,367.4
1,194530,123227,33,373.00
2,194531,123227,33,250.00
3,136763,12870648,89,365.02
4,136764,12870648,89,250.00


In [ ]:
count=1
index=0
alld_in=0
alld=pd.DataFrame(columns=["Enc_Nbr","ICD","Num_Encs"])
while index<len(icd):
    cur=icd.loc[index,"Enc_Nbr"]
    alld.loc[alld_in,"Enc_Nbr"]=cur
    nex=icd.loc[index+1,"Enc_Nbr"]
    alld.loc[alld_in,"ICD"]=icd.loc[index,"Diagnosis_Code_ID"]
    loop_count=1
    while cur==nex:
        alld.loc[alld_in,"ICD"] = alld.loc[alld_in,"ICD"] + ","+ icd.loc[index+1,"Diagnosis_Code_ID"]
        index=index+1
        nex=icd.loc[index+1,"Enc_Nbr"]
        loop_count=loop_count+1
        alld.loc[alld_in,"Num_Encs"] = loop_count
    if alld.loc[alld_in,"Num_Encs"] != loop_count :
        alld.loc[alld_in,"Num_Encs"] = count  
    alld_in=alld_in+1
    index=index+1

In [273]:
alld.shape

(64217, 4)

In [275]:
alld.to_csv('alld.csv')

In [272]:
alld=pd.read_csv(interpath+'alld.csv')

In [360]:
alld.head()

NameError: name 'alld' is not defined

In [280]:
all_enc_4=pd.merge(all_enc_3,alld, how='left',on='Enc_Nbr')

In [14]:
all_enc_4.shape

NameError: name 'all_enc_4' is not defined

Creating single ecnounter record for a person 

In [15]:
ale=pd.read_csv(interpath+'all_enc_4.csv')

In [19]:
ale.head()

,index,Enc_Nbr,Person_Nbr,Enc_Date,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic_x,BP_Diastolic_x,ICD,Num_Encs
0,0.0,12870648.0,89.0,2012-10-16 03:45:00,Former smoker,Adv BCBS HMO,0.0,26.04,140.0,140.0,88.0,"365.02,250.00",2.0
1,1.0,4126172.0,89.0,2014-03-12 15:30:00,Former smoker,Adv BCBS HMO,0.0,29.16,120.0,128.0,78.0,"362.81,365.02,250.00",3.0
2,2.0,9217142.0,89.0,2014-04-17 18:45:00,Former smoker,Adv BCBS HMO,0.0,29.16,130.0,132.0,86.0,365.02,1.0
3,3.0,14968450.0,89.0,2014-05-25 10:45:00,Former smoker,Adv BCBS HMO,0.0,26.38,130.0,78.0,62.0,"250.00,365.02,365.23,365.23",4.0
4,4.0,14537805.0,89.0,2015-03-14 15:30:00,Former smoker,Adv BCBS HMO,0.0,0.00,0.0,125.0,73.0,365.06,1.0


In [17]:
del ale["Duplicate_Exists"]
del ale["Unnamed: 0"]

KeyError: 'Duplicate_Exists'

Split Enc_Date into Enc_Yr and Enc_Time

In [ ]:
ale["Enc_Yr"]=ale.Enc_Date.str.split(" ",1,expand=True)[0]
ale["Enc_Time"]=ale.Enc_Date.str.split(" ",1,expand=True)[1]
print ale.shape
ale.columns

In [214]:
ale['Enc_Yr'] =  pd.to_datetime(ale['Enc_Yr'], format='%Y-%m-%d')

In [215]:
ale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80946 entries, 0 to 80945
Data columns (total 14 columns):
Enc_Nbr           80946 non-null float64
Person_Nbr        80946 non-null float64
Enc_Date          80946 non-null object
Smoking_Status    57718 non-null object
Primary_Payer     80946 non-null object
A1C               80946 non-null float64
BMI               80946 non-null float64
Glucose           80946 non-null float64
BP_Systolic_x     80946 non-null float64
BP_Diastolic_x    80946 non-null float64
ICD               61861 non-null object
Num_Encs          61861 non-null float64
Enc_Yr            80946 non-null datetime64[ns]
Enc_Time          80946 non-null object
dtypes: datetime64[ns](1), float64(8), object(5)
memory usage: 8.6+ MB


Split data into two tables, one with single encounters second with multiple encounters

In [210]:
lastyrcols=['Enc_Nbr', 'Person_Nbr', 'Smoking_Status',
       'Primary_Payer', 'A1C', 'BMI', 'Glucose', 'BP_Systolic',
       'BP_Diastolic','Enc_Yr', 'Enc_Time']

In [216]:
index_lsyr=0
index_num_encs=0
index_ale1=0
lastyr_encs=pd.DataFrame(columns=lastyrcols)
ale1=pd.DataFrame(columns=lastyrcols)
num_encs_total=pd.DataFrame(columns=["Person_Nbr","Total_Num_Encs","Total_Num_Enc_Lastyr"])
for name,group in ale.groupby(["Person_Nbr"]):
    num_encs_total.loc[index_num_encs,"Person_Nbr"]=group.get_value(index=0,col=1,takeable=True)
    num_encs_total.loc[index_num_encs,"Total_Num_Encs"]=len(group.get_values())
    if len(group.get_values())>1:
        recentdate=group['Enc_Yr'].max()
        recentyear=recentdate.year
        previuosyear=recentyear-1
        num_enc_lsyr=0
        for i in range(len(group.get_values())):
            enc_yr= group.get_value(index=i,col=12,takeable=True).year
            if previuosyear<=enc_yr<=recentyear:
                num_enc_lsyr=num_enc_lsyr+1
                lastyr_encs.loc[index_lsyr,"Enc_Nbr"]=group.get_value(index=i,col=0,takeable=True)
                lastyr_encs.loc[index_lsyr, 'Person_Nbr']=group.get_value(index=i,col=1,takeable=True)
                lastyr_encs.loc[index_lsyr, 'Smoking_Status']=group.get_value(index=i,col=3,takeable=True)
                lastyr_encs.loc[index_lsyr,'Primary_Payer']=group.get_value(index=i,col=4,takeable=True)
                lastyr_encs.loc[index_lsyr,'A1C']=group.get_value(index=i,col=5,takeable=True)
                lastyr_encs.loc[index_lsyr, 'BMI']=group.get_value(index=i,col=6,takeable=True)
                lastyr_encs.loc[index_lsyr, 'Glucose']=group.get_value(index=i,col=7,takeable=True)
                lastyr_encs.loc[index_lsyr, 'BP_Systolic']=group.get_value(index=i,col=8,takeable=True)
                lastyr_encs.loc[index_lsyr,'BP_Diastolic']=group.get_value(index=i,col=9,takeable=True)
                lastyr_encs.loc[index_lsyr,'Enc_Yr']=group.get_value(index=i,col=12,takeable=True)
                lastyr_encs.loc[index_lsyr,'Enc_Time']=group.get_value(index=i,col=13,takeable=True)
                index_lsyr=index_lsyr+1
        num_encs_total.loc[index_num_encs,"Total_Num_Enc_Lastyr"]=num_enc_lsyr
    else:
        ale1.loc[index_ale1,'Enc_Nbr']=group.get_value(index=0,col=0,takeable=True)
        ale1.loc[index_ale1, 'Person_Nbr']=group.get_value(index=0,col=1,takeable=True)
        ale1.loc[index_ale1, 'Smoking_Status']=group.get_value(index=0,col=3,takeable=True)
        ale1.loc[index_ale1,'Primary_Payer']=group.get_value(index=0,col=4,takeable=True)
        ale1.loc[index_ale1,'A1C']=group.get_value(index=0,col=5,takeable=True)
        ale1.loc[index_ale1, 'BMI']=group.get_value(index=0,col=6,takeable=True)
        ale1.loc[index_ale1, 'Glucose']=group.get_value(index=0,col=7,takeable=True)
        ale1.loc[index_ale1, 'BP_Systolic']=group.get_value(index=0,col=8,takeable=True)
        ale1.loc[index_ale1,'BP_Diastolic']=group.get_value(index=0,col=9,takeable=True)
        ale1.loc[index_ale1,'Enc_Yr']=group.get_value(index=0,col=12,takeable=True)
        ale1.loc[index_ale1,'Enc_Time']=group.get_value(index=0,col=13,takeable=True)
        index_ale1=index_ale1+1
        num_encs_total.loc[index_num_encs,"Total_Num_Enc_Lastyr"]=1
    index_num_encs=index_num_encs+1

Table with total number encunters and the total number encunters in the most recent last year  for each person

In [218]:
print num_encs_total.shape
num_encs_total.head()

(16087, 3)


,Person_Nbr,Total_Num_Encs,Total_Num_Enc_Lastyr
0,33,1,1
1,89,9,5
2,146,5,3
3,196,2,2
4,327,1,1


Table with single encounters

In [219]:
print ale1.shape
ale1.head()

(4545, 11)


,Enc_Nbr,Person_Nbr,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,Enc_Time
0,123227,33,Never smoker,Premier Vision Plan,8,0,130,59,39,2014-12-18 00:00:00,14:00:00
1,1.43611e+07,327,Never smoker,Medicare,0,0,121,0,0,2011-12-04 00:00:00,11:00:00
2,1.17838e+07,335,Former smoker,Blue Shield Of Illinois PPO,0,0,103,0,0,2013-10-16 00:00:00,17:45:00
3,6.0747e+06,1515,Never smoker,VOHHA,0,0,200,130,80,2012-10-01 00:00:00,15:30:00
4,8.58324e+06,1519,Former smoker,Opticare CountyCare,0,37.11,118,110,80,2014-07-01 00:00:00,17:45:00


Table with multiple encounters

In [220]:
print lastyr_encs.shape
lastyr_encs.head()

(46019, 11)


,Enc_Nbr,Person_Nbr,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,Enc_Time
0,1.45378e+07,89,Former smoker,Adv BCBS HMO,0,0,0,125,73,2015-03-14 00:00:00,15:30:00
1,1.3816e+07,89,NaN,Adv BCBS HMO,0,0,0,0,0,2015-07-11 00:00:00,05:19:32.400000
2,455661,89,Former smoker,Adv BCBS HMO,0,0,137,63,38,2015-07-14 00:00:00,10:15:00
3,1.06583e+07,89,Former smoker,Adv BCBS HMO,0,0,0,92,53,2016-10-22 00:00:00,09:15:00
4,233159,89,NaN,Adv BCBS HMO,0,0,0,0,0,2016-10-22 00:00:00,11:29:08.763000


Check that no encounter was missed

In [221]:
count=0
for index,row in num_encs_total.iterrows():
    if  row["Total_Num_Encs"]>1:
        count=count+1
print "Total num of patients with encounters"
print num_encs_total.shape[0]
print "Total num of patients with multiple encounters"
print count
print "Total num of patients with single encounters"
print num_encs_total.shape[0]-count

Total num of patients with encounters
16087
Total num of patients with multiple encounters
11542
Total num of patients with single encounters
4545


In [222]:
print len(lastyr_encs.Person_Nbr.unique())
print len(ale1.Person_Nbr.unique())
len(lastyr_encs.Person_Nbr.unique())+ len(ale1.Person_Nbr.unique())

11542
4545


16087

In [223]:
num_encs_total.to_csv("num_encs_total.csv")
ale1.to_csv("ale1.csv")
lastyr_encs.to_csv("lastyr_encs.csv")

Taking avg of numeric values 

In [303]:
columns=['A1C','BMI','Glucose','BP_Systolic','BP_Diastolic']
for i in columns:
    lastyr_encs[i]=lastyr_encs[i].convert_objects(convert_numeric=True)

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [304]:
lastyr_encs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46019 entries, 0 to 46018
Data columns (total 12 columns):
Enc_Nbr                 46019 non-null object
Person_Nbr              46019 non-null object
Smoking_Status          31318 non-null object
Primary_Payer           46019 non-null object
A1C                     46019 non-null float64
BMI                     46019 non-null float64
Glucose                 46019 non-null float64
BP_Systolic             46019 non-null float64
BP_Diastolic            46019 non-null float64
Enc_Yr                  46019 non-null object
Enc_Time                46019 non-null object
Smoking_Status_Stand    46019 non-null object
dtypes: float64(5), object(7)
memory usage: 4.6+ MB


In [305]:
lastyr_avg=lastyr_encs.groupby("Person_Nbr")[columns].mean()
lastyr_avg.shape

(11542, 5)

In [306]:
lastyr_avg.to_csv("lastyr_avg.csv")

Check for change in Smoking Status and Primary Payer

In [246]:
print len(lastyr_encs.Primary_Payer.unique())
print lastyr_encs.Primary_Payer.unique()
print len(lastyr_encs.Smoking_Status.unique())
lastyr_encs.Smoking_Status.unique()

81
['Adv BCBS HMO' 'Medicare' 'EyeQuest Family Health Network'
 'Davis Vision Plan' 'Blue Shield Of Illinois PPO'
 'No Medical Insurance Accepted' 'Medicaid HFS' 'Eye Med Vision Plan'
 'VOHHA' 'Opticare CountyCare' 'OptiCare MCO' 'CIGNA PPO'
 'United Health Care PPO' 'Opticare IlliniCare ICP'
 'Human Services Medical Record' 'BCBS Medicaid FHP'
 'March - Aetna Better Health' 'Aetna Better Health MCO' 'Adv AHC Wellcare'
 'Aetna PPO' 'Railroad Medicare' 'BCBS Medicaid ICP' 'EyeQuest CountyCare'
 'Vision Service Plan' 'Commercial' 'Aetna BH Medicaid ICP'
 'Humana Medicaid ICP' 'Cigna HS MMP/MMAI' 'Adv Cigna Health Spring HMO'
 'Meridian Medicaid ICP' 'Meridian Complete MMAI' 'Premier Vision Plan'
 'Adv Humana Health Centers' 'Humana PPO' 'BCBS Medicare MMAI' 'DHS'
 'VOHHA DEC' 'OptiCare Vision Plan' 'CCAI Medicaid'
 'Opticare IlliniCare FHP' 'ICO Faculty Staff' 'VOHHA RCVA'
 'Aetna BH MMAI Premier Plan' 'ICO Family' 'March - Molina'
 'Adv Blue Shield Of Illinois' 'Adv HMOI Health Center'


array(['Former smoker', nan, 'Never smoker', 'Current some day smoker',
       'Current every day smoker', 'Light tobacco smoker',
       'Heavy tobacco smoker', 'Unknown if ever smoked', 'Smoker',
       'Never Smoker'], dtype=object)

In [248]:
lastyr_encs.shape

(46019, 11)

In [247]:
pd.isnull(lastyr_encs.Smoking_Status).sum()

14701

Standarize the Smoking_Status

In [291]:
def stand_smoke(x):
    if pd.isnull(x):
        return 'Unknown if ever smoked'
    elif x=='Unknown if ever smoked':
        return 'Unknown if ever smoked'
    elif x=='Never Smoker':
         return 'Never smoker'
    elif x=='Never smoker':
         return 'Never smoker'
    elif x == 'Smoker' :
        return 'Smoker'
    elif x == 'Current every day smoker':
        return 'Smoker'
    elif x == 'Current some day smoker':
        return 'Smoker'
    elif x == 'Light tobacco smoker' :
        return 'Smoker'
    elif x == 'Heavy tobacco smoker':
        return 'Smoker'
    elif x=="Former smoker":
        return "Former smoker"

In [295]:
lastyr_encs["Smoking_Status_Stand"]=lastyr_encs.Smoking_Status.map(stand_smoke)
print lastyr_encs.shape
lastyr_encs.head()

(46019, 12)


,Enc_Nbr,Person_Nbr,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,Enc_Time,Smoking_Status_Stand
0,1.45378e+07,89,Former smoker,Adv BCBS HMO,0,0,0,125,73,2015-03-14 00:00:00,15:30:00,Former smoker
1,1.3816e+07,89,NaN,Adv BCBS HMO,0,0,0,0,0,2015-07-11 00:00:00,05:19:32.400000,Unknown if ever smoked
2,455661,89,Former smoker,Adv BCBS HMO,0,0,137,63,38,2015-07-14 00:00:00,10:15:00,Former smoker
3,1.06583e+07,89,Former smoker,Adv BCBS HMO,0,0,0,92,53,2016-10-22 00:00:00,09:15:00,Former smoker
4,233159,89,NaN,Adv BCBS HMO,0,0,0,0,0,2016-10-22 00:00:00,11:29:08.763000,Unknown if ever smoked


In [316]:
lastyr_encs.to_csv("lastyr_encs.csv")

In [356]:
print sum(lastyr_encs.Smoking_Status_Stand==None)
print len(lastyr_encs.Smoking_Status_Stand.unique())
lastyr_encs.Smoking_Status_Stand.unique()

0
4


array(['Former smoker', 'Unknown if ever smoked', 'Never smoker', 'Smoker'], dtype=object)

In [336]:
index=0
last_smpp=pd.DataFrame(columns=['Person_Nbr', 'Smoking_Status_Standd',"Num_SS"])
for name, group in lastyr_encs.groupby("Person_Nbr"):
    last_smpp.loc[index,"Person_Nbr"]=name
    last_smpp.loc[index,"Smoking_Status_Standd"]=group.get_value(index=0,col=11,takeable=True)
    last_smpp.loc[index,"Num_SS"]=1
    for i in range(len(group.get_values())-1):
        frsts= last_smpp.loc[index,"Smoking_Status_Standd"]
        secs= group.get_value(index=i+1,col=11,takeable=True)
        if frsts!=secs:
            if frsts=='Unknown if ever smoked':
                last_smpp.loc[index,"Smoking_Status_Standd"]==secs
                last_smpp.loc[index,"Num_SS"]=1
            elif secs=='Unknown if ever smoked':
                last_smpp.loc[index,"Smoking_Status_Standd"]==frsts
                last_smpp.loc[index,"Num_SS"]=1
            else:
                boths=frsts+","+secs
                last_smpp.loc[index,"Smoking_Status_Standd"]=boths
                last_smpp.loc[index,"Num_SS"]=(last_smpp.loc[index,"Smoking_Status_Standd"].count(",")+1)     
    index=index+1 

In [337]:
print last_smpp.shape
last_smpp.head()

(11542, 3)


,Person_Nbr,Smoking_Status_Standd,Num_SS
0,89,Former smoker,1
1,146,Former smoker,1
2,196,Never smoker,1
3,331,Never smoker,1
4,535,"Former smoker,Never smoker",2


168 people with change in Smoking_status 

In [338]:
sum(last_smpp.Num_SS > 1)

168

In [344]:
last_sm.Num_SS.unique()

array([2, 3, 4, 8, 7, 5, 9, 10, 21, 6, 14], dtype=object)

Choosing the appropriate status

In [ ]:
count=0
index=0
lastyr_ss=pd.DataFrame(columns=["Person_Nbr","Smoking_Status"])
for name,group in lastyr_encs.groupby("Person_Nbr"):
    lastyr_ss.loc[index,"Person_Nbr"]=name
    group=group.sort("Enc_Yr",ascending=False)
    if len(group.get_values())>1:
        if (group.get_value(index=0,col=11,takeable=True))==group.get_value(index=1,col=11,takeable=True):
            lastyr_ss.loc[index,"Smoking_Status"]=group.get_value(index=0,col=11,takeable=True)
        elif group.get_value(index=0,col=11,takeable=True) in ("Never smoker" , "Unknown if ever smoked"):
            lastyr_ss.loc[index,"Smoking_Status"]=group.get_value(index=1,col=11,takeable=True)
        else:
            lastyr_ss.loc[index,"Smoking_Status"]=group.get_value(index=0,col=11,takeable=True)
        index=index+1
    else:
        lastyr_ss.loc[index,"Smoking_Status"]=group.get_value(index=0,col=11,takeable=True)
        index=index+1

In [ ]:
print sum(pd.isnull(lastyr_ss.Smoking_Status))
print lastyr_ss.shape
lastyr_ss.head()

Choosing the appropriate Primary Payer

In [ ]:
count=0
index=0
lastyr_pp=pd.DataFrame(columns=["Person_Nbr","Primary_Payer","Enc_Nbr","Enc_Yr","Enc_Time"])
for name,group in lastyr_encs.groupby("Person_Nbr"):
    lastyr_pp.loc[index,"Person_Nbr"]=name
    group=group.sort("Enc_Yr",ascending=False)
    lastyr_pp.loc[index,"Enc_Nbr"]=group.get_value(index=0,col=0,takeable=True)
    lastyr_pp.loc[index,"Enc_Yr"]=group.get_value(index=0,col=9,takeable=True)
    lastyr_pp.loc[index,"Enc_Time"]=group.get_value(index=0,col=10,takeable=True)
    if pd.isnull(group.get_value(index=0,col=3,takeable=True))==False:
        lastyr_pp.loc[index,"Primary_Payer"]=group.get_value(index=0,col=3,takeable=True)
    else:
        lastyr_pp.loc[index,"Primary_Payer"]=group.get_value(index=1,col=3,takeable=True)
    index=index+1

In [ ]:
print sum(pd.isnull(lastyr_pp.Primary_Payer))
print lastyr_pp.shape
lastyr_pp.head()

Merge smoking_status and pp results

In [ ]:
print lastyr_pp.shape
lastyr_ss.shape

In [ ]:
lastyr_sspp=pd.merge(lastyr_ss,lastyr_pp,left_index=True,right_index=True)
print lastyr_sspp.shape
lastyr_sspp.head()

In [ ]:
del lastyr_sspp["Person_Nbr_y"]
lastyr_sspp=lastyr_sspp.rename(columns={"Person_Nbr_x":"Person_Nbr"})
lastyr_sspp.head()

Merge sspp with avg values

In [ ]:
lastyr_avg=lastyr_avg.reset_index()
lastyr_avg.head()

In [ ]:
lastyr_all=pd.merge(lastyr_sspp,lastyr_avg,left_index=True,right_index=True)
print lastyr_all.shape
lastyr_all.head()

In [357]:
del lastyr_all["Person_Nbr_y"]
lastyr_all=lastyr_all.rename(columns={"Person_Nbr_x":"Person_Nbr"})
lastyr_all.head()

NameError: name 'lastyr_all' is not defined

Merge single and multiple encounter averages

In [ ]:
print ale1.shape
lastyr_all.shape

In [ ]:
single_enc_per_person=pd.merge(ale1,lastyr_all,how="outer")
print single_enc_per_person.shape
single_enc_per_person.head()

In [ ]:
len(single_enc_per_person.Person_Nbr.drop_duplicates())

In [ ]:
single_enc_per_person.to_csv("single_enc_per_person.csv")

Add ICDs

In [217]:
cols=['Enc_Nbr', 'Person_Nbr', 'ICD', 'Num_Encs']
icds=ale[cols]
print icds.shape
print icds.info()
print icds.head()

(80946, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80946 entries, 0 to 80945
Data columns (total 4 columns):
Enc_Nbr       80946 non-null float64
Person_Nbr    80946 non-null float64
ICD           61861 non-null object
Num_Encs      61861 non-null float64
dtypes: float64(3), object(1)
memory usage: 2.5+ MB
None
      Enc_Nbr  Person_Nbr                          ICD  Num_Encs
0  12870648.0        89.0                365.02,250.00       2.0
1   4126172.0        89.0         362.81,365.02,250.00       3.0
2   9217142.0        89.0                       365.02       1.0
3  14968450.0        89.0  250.00,365.02,365.23,365.23       4.0
4  14537805.0        89.0                       365.06       1.0


In [361]:
print "total rows with unique enc_nbr"
print ale.shape[0]
print "uni prsn_nbr"
print len(ale.Person_Nbr.drop_duplicates())
print "icds available"
print sum(pd.isnull(ale.ICD)==False)

total rows with unique enc_nbr
80946
uni prsn_nbr
16087
icds available
61861


In [386]:
index=0
icd_per_person=pd.DataFrame(columns=["Person_Nbr","ICD"])
for name,group in icds.groupby("Person_Nbr"):
    icd_per_person.loc[index,"Person_Nbr"]=name
    a=group.get_value(index=0,col=2,takeable=True)
    icd_per_person.loc[index,"ICD"]=str(group.get_value(index=0,col=2,takeable=True))
    if len(group.get_values())>1:
        for i in range(len(group.get_values())-1):
            i+=1
            icd_per_person.loc[index,"ICD"] = icd_per_person.loc[index,"ICD"] + ","+ str(group.get_value(index=i,col=2,takeable=True))
    index=index+1

In [387]:
print icd_per_person.shape
icd_per_person.head()

(16087, 2)


,Person_Nbr,ICD
0,33,"367.4,373.00,250.00"
1,89,"365.02,250.00,362.81,365.02,250.00,365.02,250...."
2,146,"366.19,374.87,250.00,250.00,366.19,250.00,366...."
3,196,"H10.45,H35.40,H04.123,H52.4,nan"
4,327,"367.4,365.01,365.01,367.4,367.4"


In [388]:
icd_per_person.to_csv("icd_per_person.csv")

Merge single_person and icd_per_person

In [389]:
print single_enc_per_person.shape
icd_per_person.shape

(16087, 11)


(16087, 2)

In [390]:
single_person_wicd=pd.merge(single_enc_per_person,icd_per_person,left_index=True,right_index=True)
print single_person_wicd.shape
single_person_wicd.head()

(16087, 13)


,Enc_Nbr,Person_Nbr_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,Enc_Time,Person_Nbr_y,ICD
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,14:00:00,33,"367.4,373.00,250.00"
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,11:00:00,89,"365.02,250.00,362.81,365.02,250.00,365.02,250...."
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,17:45:00,146,"366.19,374.87,250.00,250.00,366.19,250.00,366...."
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,15:30:00,196,"H10.45,H35.40,H04.123,H52.4,nan"
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,17:45:00,327,"367.4,365.01,365.01,367.4,367.4"


In [391]:
del single_person_wicd["Person_Nbr_y"]
single_person_wicd=single_person_wicd.rename(columns={"Person_Nbr_x":"Person_Nbr"})
single_person_wicd.head()

,Enc_Nbr,Person_Nbr,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,Enc_Time,ICD
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,14:00:00,"367.4,373.00,250.00"
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,11:00:00,"365.02,250.00,362.81,365.02,250.00,365.02,250...."
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,17:45:00,"366.19,374.87,250.00,250.00,366.19,250.00,366...."
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,15:30:00,"H10.45,H35.40,H04.123,H52.4,nan"
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,17:45:00,"367.4,365.01,365.01,367.4,367.4"


In [392]:
single_person_wicd.to_csv("single_person_wicd.csv")

Adding Macula Readings

In [364]:
macula=pd.read_pickle(path+'macula_findings_for_Enc.pickle')

In [365]:
print macula.shape
print len(macula.Enc_Nbr.drop_duplicates())
print len(macula.Person_Nbr.drop_duplicates())
macula.head()

(80946, 29)
80946
16087


,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,OD_Macula2_Modifer,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
74595,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54456,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54457,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat (-)DME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54458,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54459,71d10fa4-3b0d-fa52-cfbd-ffd0f3d3532d,14968450,2014-05-25 10:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [366]:
macula=macula.reset_index()

In [367]:
m1=macula.dropna(subset=["OD_Macula1_Finding"])
print m1.shape
m1.head()

(39940, 30)


,index,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,74595,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54456,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54457,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat (-)DME,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54458,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54463,831d44f9-4ab7-8df8-8757-bb1eebd14e6b,10658304,2016-10-22 09:15:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
len(m1.Person_Nbr.drop_duplicates())

15409

Choose the most recent findings

In [369]:
a=m1.loc[m1.groupby(['Person_Nbr'])['Enc_Date'].idxmax()]
print a.shape
a.head()

(15409, 30)


,index,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,74595,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54463,831d44f9-4ab7-8df8-8757-bb1eebd14e6b,10658304,2016-10-22 09:15:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,44553,63ecc259-208f-7ef2-b0f8-acf25e23c264,5786173,2016-04-05 10:15:00,ca793eec-f101-2505-42e6-57fefc6baf14,146,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,81657,918e81a9-5ea3-933a-6c73-219c07ffb733,14534672,2016-10-05 00:45:00,3afb193c-dbf7-9f64-5f95-447f81cd0bdf,196,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,79224,2bad0952-05f8-acc5-35f2-3c7ea950943d,3610887,2016-01-19 07:45:00,8101be56-67f1-cc74-49d0-9eb3f721cdc6,331,Large ME,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [375]:
temp = pd.DatetimeIndex(a.Enc_Date)
a["Enc_Yr"] = temp.date
a['Enc_Time'] = temp.time
print a.shape
a.columns

(15409, 32)


Index([u'index', u'Enc_ID', u'Enc_Nbr', u'Enc_Date', u'Person_ID',
       u'Person_Nbr', u'OD_Macula1_Finding', u'OD_Macula1_Modifer',
       u'OD_Macula1_Location', u'OD_Macula2_Finding', u'OD_Macula2_Modifer',
       u'OD_Macula2_Location', u'OD_Macula3_Finding', u'OD_Macula3_Modifer',
       u'OD_Macula3_Location', u'OD_Macula4_Finding', u'OD_Macula4_Modifer',
       u'OD_Macula4_Location', u'OS_Macula1_Finding', u'OS_Macula1_Modifer',
       u'OS_Macula1_Location', u'OS_Macula2_Finding', u'OS_Macula2_Modifer',
       u'OS_Macula2_Location', u'OS_Macula3_Finding', u'OS_Macula3_Modifer',
       u'OS_Macula3_Location', u'OS_Macula4_Finding', u'OS_Macula4_Modifer',
       u'OS_Macula4_Location', u'Enc_Yr', u'Enc_Time'],
      dtype='object')

In [377]:
cols=['Enc_ID','Enc_Nbr', 'Enc_Yr', 'Enc_Time',
       u'Person_Nbr', u'OD_Macula1_Finding', u'OD_Macula1_Modifer',
       u'OD_Macula1_Location', u'OD_Macula2_Finding', u'OD_Macula2_Modifer',
       u'OD_Macula2_Location', u'OD_Macula3_Finding', u'OD_Macula3_Modifer',
       u'OD_Macula3_Location', u'OD_Macula4_Finding', u'OD_Macula4_Modifer',
       u'OD_Macula4_Location', u'OS_Macula1_Finding', u'OS_Macula1_Modifer',
       u'OS_Macula1_Location', u'OS_Macula2_Finding', u'OS_Macula2_Modifer',
       u'OS_Macula2_Location', u'OS_Macula3_Finding', u'OS_Macula3_Modifer',
       u'OS_Macula3_Location', u'OS_Macula4_Finding', u'OS_Macula4_Modifer',
       u'OS_Macula4_Location']
mac=a[cols]
print mac.shape
mac.head()

(15409, 29)


,Enc_ID,Enc_Nbr,Enc_Yr,Enc_Time,Person_Nbr,OD_Macula1_Finding,OD_Macula1_Modifer,OD_Macula1_Location,OD_Macula2_Finding,OD_Macula2_Modifer,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18,14:00:00,33,flat no hemorrhages exudates or pigmentary cha...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,831d44f9-4ab7-8df8-8757-bb1eebd14e6b,10658304,2016-10-22,09:15:00,89,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,63ecc259-208f-7ef2-b0f8-acf25e23c264,5786173,2016-04-05,10:15:00,146,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,918e81a9-5ea3-933a-6c73-219c07ffb733,14534672,2016-10-05,00:45:00,196,flat no hemorrhages exudates pigmentary change...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2bad0952-05f8-acc5-35f2-3c7ea950943d,3610887,2016-01-19,07:45:00,331,Large ME,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [379]:
mac=mac.reset_index()

Merge with single_person_wicd

In [393]:
print mac.shape
single_person_wicd.shape

(15409, 30)


(16087, 12)

In [396]:
single_person_wmac=pd.merge(single_person_wicd,mac,left_index=True,right_index=True,how="left")
print single_person_wmac.shape
single_person_wmac.head()

(16087, 42)


,Enc_Nbr_x,Person_Nbr_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr_x,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [397]:
single_person_wmac.columns

Index([u'Enc_Nbr_x', u'Person_Nbr_x', u'Smoking_Status', u'Primary_Payer',
       u'A1C', u'BMI', u'Glucose', u'BP_Systolic', u'BP_Diastolic',
       u'Enc_Yr_x', u'Enc_Time_x', u'ICD', u'index', u'Enc_ID', u'Enc_Nbr_y',
       u'Enc_Yr_y', u'Enc_Time_y', u'Person_Nbr_y', u'OD_Macula1_Finding',
       u'OD_Macula1_Modifer', u'OD_Macula1_Location', u'OD_Macula2_Finding',
       u'OD_Macula2_Modifer', u'OD_Macula2_Location', u'OD_Macula3_Finding',
       u'OD_Macula3_Modifer', u'OD_Macula3_Location', u'OD_Macula4_Finding',
       u'OD_Macula4_Modifer', u'OD_Macula4_Location', u'OS_Macula1_Finding',
       u'OS_Macula1_Modifer', u'OS_Macula1_Location', u'OS_Macula2_Finding',
       u'OS_Macula2_Modifer', u'OS_Macula2_Location', u'OS_Macula3_Finding',
       u'OS_Macula3_Modifer', u'OS_Macula3_Location', u'OS_Macula4_Finding',
       u'OS_Macula4_Modifer', u'OS_Macula4_Location'],
      dtype='object')

In [398]:
del single_person_wmac["Person_Nbr_y"]
del single_person_wmac["Enc_Time_y"]
single_person_wmac=single_person_wmac.rename(columns={"Person_Nbr_x":"Person_Nbr","Enc_Nbr_x":"Enc_Nbr","Enc_Nbr_y":"Enc_Nbr_mac","Enc_Yr_x":"Enc_Yr","Enc_Time_x":"Enc_Time","Enc_Yr_y":"Enc_Yr_mac"})
print single_person_wmac.shape
single_person_wmac.head()

(16087, 40)


,Enc_Nbr,Person_Nbr,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,...,OS_Macula1_Location,OS_Macula2_Finding,OS_Macula2_Modifer,OS_Macula2_Location,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [400]:
single_person_wmac.to_csv("single_person_wmac.csv")

Add SL_Lens Readings

In [102]:
sl_lens=pd.read_pickle(path+'SL_Lens_for_Enc.pickle')
print sl_lens.shape
print len(sl_lens.Enc_Nbr.drop_duplicates())
print len(sl_lens.Person_Nbr.drop_duplicates())
sl_lens.head()

(80946, 7)
80946
16087


,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_SL_LENS,OS_SL_LENS
74679,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC
54483,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+ with trace ACC changes
54484,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
54485,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
54486,71d10fa4-3b0d-fa52-cfbd-ffd0f3d3532d,14968450,2014-05-25 10:45:00,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,NaN,NaN


In [103]:
sl_lens.to_csv('sl_lens.csv')

In [104]:
sl_lens=sl_lens.reset_index()

In [404]:
sl=sl_lens.dropna(subset=["OD_SL_LENS"])
print sl.shape
sl.head()

(54383, 8)


,Unnamed: 0,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_SL_LENS,OS_SL_LENS
0,74679,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00.000,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC
1,54483,a6d9d991-4ff4-79b5-304d-f1b2bff9d6d0,12870648,2012-10-16 03:45:00.000,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+ with trace ACC changes
2,54484,fe5eb87c-39b3-4314-0172-4642d51de417,4126172,2014-03-12 15:30:00.000,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
3,54485,ea3c2622-6d1d-53ab-ddf0-9764927f1507,9217142,2014-04-17 18:45:00.000,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,nuclear sclerosis 1+,nuclear sclerosis 1+
5,54487,6073586c-f74c-dda4-1a77-7ae779034993,14537805,2015-03-14 15:30:00.000,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus


In [405]:
sl.to_csv("sl.csv")

In [406]:
len(sl.Person_Nbr.drop_duplicates())

15783

Choose the most recent findings

In [415]:
sl["Enc_Yr"]=sl.Enc_Date.str.split(" ",1,expand=True)[0]
sl["Enc_Time"]=sl.Enc_Date.str.split(" ",1,expand=True)[1]
sl['Enc_Yr'] =  pd.to_datetime(ale['Enc_Yr'], format='%Y-%m-%d')
print sl.shape
sl.columns

C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(54383, 10)


C:\Users\mom1\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Index([u'Unnamed: 0', u'Enc_ID', u'Enc_Nbr', u'Enc_Date', u'Person_ID',
       u'Person_Nbr', u'OD_SL_LENS', u'OS_SL_LENS', u'Enc_Yr', u'Enc_Time'],
      dtype='object')

In [416]:
b=sl.loc[sl.groupby(['Person_Nbr'])['Enc_Yr'].idxmax()]
print b.shape
b.head()

(15783, 10)


,Unnamed: 0,Enc_ID,Enc_Nbr,Enc_Date,Person_ID,Person_Nbr,OD_SL_LENS,OS_SL_LENS,Enc_Yr,Enc_Time
0,74679,97825c51-4462-eade-1c1d-2baa3400c033,123227,2014-12-18 14:00:00.000,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC,2012-10-16,14:00:00.000
7,54488,d1e94fb5-ecdd-b363-7e88-a558e01ba87e,455661,2015-07-14 10:15:00.000,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus,2016-10-22,10:15:00.000
14,44579,63ecc259-208f-7ef2-b0f8-acf25e23c264,5786173,2016-04-05 10:15:00.000,ca793eec-f101-2505-42e6-57fefc6baf14,146,2 + NS (undilated),2 + NS (undilated),2016-10-05,10:15:00.000
15,81629,918e81a9-5ea3-933a-6c73-219c07ffb733,14534672,2016-10-05 00:45:00.000,3afb193c-dbf7-9f64-5f95-447f81cd0bdf,196,1+ NS,1+ NS,2016-10-10,00:45:00.000
18,79263,794f5f00-407f-33d1-fc30-fce766ac075c,2071158,2015-12-01 08:15:00.000,8101be56-67f1-cc74-49d0-9eb3f721cdc6,331,vacuoles,vacuoles,2015-12-19,08:15:00.000


In [417]:
b=b.reset_index()

Merge with single_person_wmac

In [418]:
print b.shape
single_person_wmac.shape

(15783, 11)


(16087, 40)

In [419]:
single_person_wsl=pd.merge(single_person_wmac,b,left_index=True,right_index=True,how="left")
print single_person_wsl.shape
single_person_wsl.head()

(16087, 51)


,Enc_Nbr_x,Person_Nbr_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr_x,...,Unnamed: 0,Enc_ID_y,Enc_Nbr_y,Enc_Date,Person_ID,Person_Nbr_y,OD_SL_LENS,OS_SL_LENS,Enc_Yr_y,Enc_Time_y
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,...,74679.0,97825c51-4462-eade-1c1d-2baa3400c033,123227.0,2014-12-18 14:00:00.000,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33.0,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC,2012-10-16,14:00:00.000
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,...,54488.0,d1e94fb5-ecdd-b363-7e88-a558e01ba87e,455661.0,2015-07-14 10:15:00.000,adca6fa4-e7d4-d7f8-cf41-27056662d84b,89.0,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus,2016-10-22,10:15:00.000
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,...,44579.0,63ecc259-208f-7ef2-b0f8-acf25e23c264,5786173.0,2016-04-05 10:15:00.000,ca793eec-f101-2505-42e6-57fefc6baf14,146.0,2 + NS (undilated),2 + NS (undilated),2016-10-05,10:15:00.000
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,...,81629.0,918e81a9-5ea3-933a-6c73-219c07ffb733,14534672.0,2016-10-05 00:45:00.000,3afb193c-dbf7-9f64-5f95-447f81cd0bdf,196.0,1+ NS,1+ NS,2016-10-10,00:45:00.000
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,...,79263.0,794f5f00-407f-33d1-fc30-fce766ac075c,2071158.0,2015-12-01 08:15:00.000,8101be56-67f1-cc74-49d0-9eb3f721cdc6,331.0,vacuoles,vacuoles,2015-12-19,08:15:00.000


In [420]:
single_person_wsl.columns

Index([u'Enc_Nbr_x', u'Person_Nbr_x', u'Smoking_Status', u'Primary_Payer',
       u'A1C', u'BMI', u'Glucose', u'BP_Systolic', u'BP_Diastolic',
       u'Enc_Yr_x', u'Enc_Time_x', u'ICD', u'index_x', u'Enc_ID_x',
       u'Enc_Nbr_mac', u'Enc_Yr_mac', u'OD_Macula1_Finding',
       u'OD_Macula1_Modifer', u'OD_Macula1_Location', u'OD_Macula2_Finding',
       u'OD_Macula2_Modifer', u'OD_Macula2_Location', u'OD_Macula3_Finding',
       u'OD_Macula3_Modifer', u'OD_Macula3_Location', u'OD_Macula4_Finding',
       u'OD_Macula4_Modifer', u'OD_Macula4_Location', u'OS_Macula1_Finding',
       u'OS_Macula1_Modifer', u'OS_Macula1_Location', u'OS_Macula2_Finding',
       u'OS_Macula2_Modifer', u'OS_Macula2_Location', u'OS_Macula3_Finding',
       u'OS_Macula3_Modifer', u'OS_Macula3_Location', u'OS_Macula4_Finding',
       u'OS_Macula4_Modifer', u'OS_Macula4_Location', u'index_y',
       u'Unnamed: 0', u'Enc_ID_y', u'Enc_Nbr_y', u'Enc_Date', u'Person_ID',
       u'Person_Nbr_y', u'OD_SL_LENS', u'OS_SL_

In [424]:
del single_person_wsl["Person_Nbr_y"]
del single_person_wsl["Enc_Time_y"]
del single_person_wsl['index_x']
del single_person_wsl['index_y']
del single_person_wsl['Enc_ID_x']
del single_person_wsl['Enc_ID_y']
del single_person_wsl['Unnamed: 0']
del single_person_wsl['Enc_Date']
del single_person_wsl['Person_ID']
single_person_wsl=single_person_wsl.rename(columns={"Person_Nbr_x":"Person_Nbr","Enc_Nbr_x":"Enc_Nbr","Enc_Nbr_y":"Enc_Nbr_sl",
                                                      "Enc_Yr_x":"Enc_Yr","Enc_Time_x":"Enc_Time","Enc_Yr_y":"Enc_Yr_sl"})
print single_person_wsl.shape
single_person_wsl.head()

(16087, 42)


,Enc_Nbr,Person_Nbr,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,...,OS_Macula3_Finding,OS_Macula3_Modifer,OS_Macula3_Location,OS_Macula4_Finding,OS_Macula4_Modifer,OS_Macula4_Location,Enc_Nbr_sl,OD_SL_LENS,OS_SL_LENS,Enc_Yr_sl
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,123227.0,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC,2012-10-16
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,455661.0,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus,2016-10-22
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,5786173.0,2 + NS (undilated),2 + NS (undilated),2016-10-05
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,14534672.0,1+ NS,1+ NS,2016-10-10
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,2071158.0,vacuoles,vacuoles,2015-12-19


In [425]:
single_person_wsl.to_csv("single_person_wsl.csv")

Adding family hist

In [112]:
family_hist=pd.read_pickle(path+'family_hist_list.pickle')
print family_hist.shape
print len(family_hist.Person_Nbr.drop_duplicates())
family_hist.head()

(59203, 7)
14044


,Person_ID,Person_Nbr,Date_Created,Code,Code_System,Family_History,Relation
47872,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:15.980,160347007,SNOMED,Glaucoma,Father
47869,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:26.897,160274005,SNOMED,No history of Diabetes mellitus,Mother
47867,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:28.590,160274005,SNOMED,No history of Diabetes mellitus,Father
47871,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:29.563,160267000,SNOMED,No history of Glaucoma,Mother
47868,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 15:55:35.837,439724007,SNOMED,Heart disease,Brother


In [426]:
fam_his=pd.read_csv(interpath+'family_hist-csv (1).csv')
print fam_his.shape
print len(fam_his.Person_Nbr.drop_duplicates())
fam_his.head()

(59186, 9)
14044


,Column,index,Person_ID,Person_Nbr,Date_Created,Code,Code_System,Family_History,Relation
0,0,47872,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:15.980,160347007.0,SNOMED,Glaucoma,Parents
1,1,47869,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:26.897,160274005.0,SNOMED,No history of Diabetes mellitus,Parents
2,2,47867,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:28.590,160274005.0,SNOMED,No history of Diabetes mellitus,Parents
3,3,47871,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:29.563,160267000.0,SNOMED,No history of Glaucoma,Parents
4,4,47868,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 15:55:35.837,439724007.0,SNOMED,Heart Disease,Siblings


In [427]:
del fam_his["Column"]

In [428]:
sum(pd.isnull(fam_his.Code))

1192

In [429]:
fam_his=fam_his.dropna(subset=["Code"])
print fam_his.shape
print len(fam_his.Person_Nbr.drop_duplicates())
fam_his.head()

(57994, 8)
13907


,index,Person_ID,Person_Nbr,Date_Created,Code,Code_System,Family_History,Relation
0,47872,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:15.980,160347007.0,SNOMED,Glaucoma,Parents
1,47869,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:26.897,160274005.0,SNOMED,No history of Diabetes mellitus,Parents
2,47867,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:28.590,160274005.0,SNOMED,No history of Diabetes mellitus,Parents
3,47871,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 14:47:29.563,160267000.0,SNOMED,No history of Glaucoma,Parents
4,47868,80d3df88-dddf-5ad3-7cc1-b7b1ac6151fa,33,2014-12-18 15:55:35.837,439724007.0,SNOMED,Heart Disease,Siblings


In [430]:
print len(fam_his.Relation.unique())
fam_his.Relation.unique()

6


array(['Parents', 'Siblings', 'Relatives', 'Grandparents', 'Children',
       'Spouse'], dtype=object)

In [433]:
cols=["Person_Nbr",'Relation',"Code","Family_History"]
b=fam_his[cols]
b.head()

,Person_Nbr,Relation,Code,Family_History
0,33,Parents,160347007.0,Glaucoma
1,33,Parents,160274005.0,No history of Diabetes mellitus
2,33,Parents,160274005.0,No history of Diabetes mellitus
3,33,Parents,160267000.0,No history of Glaucoma
4,33,Siblings,439724007.0,Heart Disease


In [447]:
count=0
index=0
e=pd.DataFrame(columns=['Person_Nbr', 'Relation',"Code","Family_History"])
for name, group in b.groupby(['Person_Nbr', 'Relation']):
    if len(e)==0:
        e.loc[index,"Person_Nbr"]=name[0]
        e.loc[index,"Relation"]=(name[1])
        e.loc[index,"Code"]=str(group.get_value(index=0,col=2,takeable=True))
        e.loc[index,"Family_History"]=str(group.get_value(index=0,col=3,takeable=True))
        for i in range(len(group.get_values())-1):
            i+=1
            e.loc[index,"Code"]=str(e.loc[index,"Code"])+','+str(group.get_value(index=i,col=2,takeable=True))
            e.loc[index,"Family_History"]=str(e.loc[index,"Family_History"])+','+str(group.get_value(index=i,col=3,takeable=True))
        e.loc[index,"Code"]=e.loc[index,"Code"]+';'
        e.loc[index,"Family_History"]=e.loc[index,"Family_History"]+';'
        index=index+1
    elif (e.loc[index-1,"Person_Nbr"]==name[0] ):
        e.loc[index-1,"Relation"]=e.loc[index-1,"Relation"]+';'+(name[1])
        for i in range(len(group.get_values())-1):
            e.loc[index-1,"Code"]=str(e.loc[index-1,"Code"])+','+str(group.get_value(index=i,col=2,takeable=True))
            e.loc[index-1,"Family_History"]=str(e.loc[index-1,"Family_History"])+','+str(group.get_value(index=i,col=3,takeable=True))
        e.loc[index-1,"Code"]=e.loc[index-1,"Code"]+';'
        e.loc[index-1,"Family_History"]=e.loc[index-1,"Family_History"]+';'
    else:
        e.loc[index,"Person_Nbr"]=name[0]
        e.loc[index,"Relation"]=(name[1])
        e.loc[index,"Code"]=str(group.get_value(index=0,col=2,takeable=True))
        e.loc[index,"Family_History"]=str(group.get_value(index=0,col=3,takeable=True))
        for i in range(len(group.get_values())-1):
            i+=1
            e.loc[index,"Code"]=str(e.loc[index,"Code"])+','+str(group.get_value(index=i,col=2,takeable=True))
            e.loc[index,"Family_History"]=str(e.loc[index,"Family_History"])+','+str(group.get_value(index=i,col=3,takeable=True))
        e.loc[index,"Code"]=e.loc[index,"Code"]+';'
        e.loc[index-1,"Family_History"]=e.loc[index-1,"Family_History"]+';'
        index=index+1

In [449]:
print e.shape
e.head()

(13907, 4)


,Person_Nbr,Relation,Code,Family_History
0,33,Parents;Siblings,"160347007.0,160274005.0,160274005.0,160267000....","Glaucoma,No history of Diabetes mellitus,No hi..."
1,89,Parents;Relatives,"416855002.0,160274005.0,160267000.0,160267000.0;;","Diabetes,No history of Diabetes mellitus,No hi..."
2,146,Parents,"416855002.0,275134007.0,160357008.0,160267000....","Diabetes,Arthritis,HBP,No history of Glaucoma,..."
3,196,Parents,"416855002.0,160267000.0,160274005.0,160267000.0;","Diabetes,No history of Glaucoma,No history of ..."
4,331,Parents;Siblings,"160274005.0,160267000.0,160274005.0,160267000.0;;","No history of Diabetes mellitus,No history of ..."


In [451]:
e.to_csv("fam_his_per_person.csv")

Merge with single_person_wsl

In [450]:
print e.shape
single_person_wsl.shape

(13907, 4)


(16087, 42)

In [10]:
single_person_all=pd.merge(single_person_wsl,e,left_index=True,right_index=True,how="left")
print single_person_all.shape
single_person_all.head()

(16087, 46)


,Enc_Nbr,Person_Nbr_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,...,OS_Macula4_Modifer,OS_Macula4_Location,Enc_Nbr_sl,OD_SL_LENS,OS_SL_LENS,Enc_Yr_sl,Person_Nbr_y,Relation,Code,Family_History
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,...,NaN,NaN,123227.0,1+ NS 1-2+ ACC,1+ NS 1-2+ ACC,2012-10-16,33.0,Parents;Siblings,"160347007.0,160274005.0,160274005.0,160267000....","Glaucoma,No history of Diabetes mellitus,No hi..."
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,...,NaN,NaN,455661.0,clear lens capsule cortex and nucleus,clear lens capsule cortex and nucleus,2016-10-22,89.0,Parents;Relatives,"416855002.0,160274005.0,160267000.0,160267000.0;;","Diabetes,No history of Diabetes mellitus,No hi..."
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,...,NaN,NaN,5786173.0,2 + NS (undilated),2 + NS (undilated),2016-10-05,146.0,Parents,"416855002.0,275134007.0,160357008.0,160267000....","Diabetes,Arthritis,HBP,No history of Glaucoma,..."
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,...,NaN,NaN,14534672.0,1+ NS,1+ NS,2016-10-10,196.0,Parents,"416855002.0,160267000.0,160274005.0,160267000.0;","Diabetes,No history of Glaucoma,No history of ..."
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,...,NaN,NaN,2071158.0,vacuoles,vacuoles,2015-12-19,331.0,Parents;Siblings,"160274005.0,160267000.0,160274005.0,160267000.0;;","No history of Diabetes mellitus,No history of ..."


In [11]:
del single_person_all["Person_Nbr_y"]
single_person_all=single_person_all.rename(columns={"Person_Nbr_x":"Person_Nbr"})
single_person_all.to_csv("single_person_all.csv")

In [12]:
single_person_all.to_csv("single_person_all.csv")

Adding demographics 

In [13]:
demographics = pd.read_pickle(path + 'demographics.pickle') 

In [14]:
print demographics.shape
print len(demographics.Person_Nbr.drop_duplicates())
demographics.head()

(17019, 8)
17019


,﻿Person_ID,Person_Nbr,DOB,Gender,Race,Ethnicity,Zip,Age_Censored
14132,18405351-AC64-46A2-A003-8F7889351A13,33,1948-10-15,F,Black/African American (Not Hispanic),Declined to specify,60616,None
8747,4ACD51E8-4A9B-4AA6-B635-166ADA5EF02E,89,1955-09-04,M,Declined to specify,Declined to specify,60652,None
2129,DF25CE0F-E45A-44D2-BE43-F49FE4F5E280,112,1932-11-19,F,NaN,NaN,606372549,None
6591,7027DD58-6465-496C-880A-A6E825C155A8,146,1948-12-19,F,Declined to specify,Declined to specify,60615,None
16909,395DD5E7-45F9-456E-A127-9AB929E872F7,196,1955-02-21,F,Declined to specify,Declined to specify,60615,None


In [15]:
demographics=demographics.reset_index()

Finding Age from DOB

In [18]:
temp = pd.DatetimeIndex(all_enc.Enc_Date)
max_date=temp.date.max()
max_date

datetime.date(2016, 11, 3)

In [19]:
demographics["Age"]=demographics.DOB.apply(lambda x:relativedelta(max_date,x).years)

Fixing the zip code column

In [20]:
sum(demographics.Zip.apply(lambda x: "True" if len(x)>5 else "False")=="True")

2012

In [21]:
demographics["Zipn"]=demographics.Zip.apply(lambda x: x[0:5])

Standardize the Race and Ethinicity

In [22]:
print len(demographics.Race.unique())
print demographics.Race.unique()
print len(demographics.Ethnicity.unique())
demographics.Ethnicity.unique()

31
['Black/African American (Not Hispanic)' 'Declined to specify' nan
 'Black or African American' 'Hispanic' 'Indian' 'Asian'
 'Hispanic Or Latino (All Races)' 'White (Not Hispanic / Latino)' 'White'
 'Native American Indian' 'Alaskan Native' 'Multi-racial'
 'American Indian or Alaska Native' 'Multiracial' 'Other Race (Jamaican)'
 'Other Pacific Islander (Not Hawaiian)' 'Greek'
 'American Indian/Alaskan Native' 'Other Race' 'Hawaiian'
 'Unknown/Not Reported' 'More than one race'
 'Native Hawaiian or Other Pacific Islander' 'AA' 'Chinese'
 'American Indian' 'African American' 'Caucasian' 'Pakistani' 'Moroccan']
16


array(['Declined to specify', nan, 'African American',
       'Not Hispanic or Latino', 'Unknown Ethnicity', 'Hispanic or Latino',
       'Unknown / Not Reported', 'african american', 'Chinese', 'Black',
       'African', 'Caucasion and black', 'Pt declines to answer',
       'Black/AA', 'white', 'Black/ AA'], dtype=object)

In [23]:
def stand_race(x):
    if pd.isnull(x):
        return 'Unknown'
    elif x=='African American':
        return 'Black or African American'
    elif x=='Black or African American':
        return'Black or African American'
    elif x=='Black/African American (Not Hispanic)':
        return 'Black or African American'
    elif x=='American Indian or Alaska Native':
        return 'American Indian or Alaska Native'
    elif x==        'American Indian/Alaskan Native':
        return'American Indian or Alaska Native'
    elif x==        'American Indian':
        return 'American Indian or Alaska Native'
    elif x==        'Native American Indian':
        return'American Indian or Alaska Native'
    elif x==        'Alaskan Native':
        return 'American Indian or Alaska Native'
    elif x==       'Asian':
        return'Asian'
    elif x==       'Chinese':
        return'Asian'
    elif x==       'Indian':
        return'Asian'
    elif x==       'Caucasian':
        return'White'
    elif x==       'White (Not Hispanic / Latino)':
        return'White, not Hispanic or Latino'
    elif x==        'White':
        return'White'
    elif x==       'Declined to specify':
        return'Unknown'
    elif x==      'Unknown/Not Reported':
        return'Unknown'
    elif x==      'Greek':
        return'White'
    elif x==      'Native Hawaiian or Other Pacific Islander':
        return'Native Hawaiian and Other Pacific Islander'
    elif x==      'Hawaiian':
        return'Native Hawaiian and Other Pacific Islander'
    elif x==      'Other Pacific Islander (Not Hawaiian)':
        return'Native Hawaiian and Other Pacific Islander'
    elif x==      'Hispanic Or Latino (All Races)':
        return'Hispanic or Latino'
    elif x==       'Hispanic':
        return'Hispanic or Latino'
    elif x==       'More than one race':
        return'Two or More Races'
    elif x==       'Multiracial':
        return'Two or More Races'
    elif x==       'Multi-racial':
        return'Two or More Races'
    elif x=='Moroccan':
        return'White'
    elif x==       'Other Race':
        return'Other Race'
    elif x==       'Other Race (Jamaican)':
        return'Other Race'

In [24]:
demographics["Racen"]=demographics.Race.map(stand_race)

In [25]:
print len(demographics.Racen.unique())
demographics.Racen.unique()

11


array(['Black or African American', 'Unknown', 'Hispanic or Latino',
       'Asian', 'White, not Hispanic or Latino', 'White',
       'American Indian or Alaska Native', 'Two or More Races',
       'Other Race', 'Native Hawaiian and Other Pacific Islander', None], dtype=object)

In [26]:
cols=["Person_Nbr","Age","Zipn","Gender","Racen"]
demo=demographics[cols]
print demo.shape
demo.head()

(17019, 5)


,Person_Nbr,Age,Zipn,Gender,Racen
0,33,68,60616,F,Black or African American
1,89,61,60652,M,Unknown
2,112,83,60637,F,Unknown
3,146,67,60615,F,Unknown
4,196,61,60615,F,Unknown


Merge with single_person_all

In [27]:
print demo.shape
single_person_all.shape

(17019, 5)


(16087, 45)

In [28]:
single_person_all1=pd.merge(single_person_all,demo,left_index=True,right_index=True,how="left")
print single_person_all1.shape
single_person_all1.head()

(16087, 50)


,Enc_Nbr,Person_Nbr_x,Smoking_Status,Primary_Payer,A1C,BMI,Glucose,BP_Systolic,BP_Diastolic,Enc_Yr,...,OS_SL_LENS,Enc_Yr_sl,Relation,Code,Family_History,Person_Nbr_y,Age,Zipn,Gender,Racen
0,123227.0,33.0,Never smoker,Premier Vision Plan,8.0,0.00,130.0,59.0,39.0,2014-12-18 00:00:00,...,1+ NS 1-2+ ACC,2012-10-16,Parents;Siblings,"160347007.0,160274005.0,160274005.0,160267000....","Glaucoma,No history of Diabetes mellitus,No hi...",33,68,60616,F,Black or African American
1,14361139.0,327.0,Never smoker,Medicare,0.0,0.00,121.0,0.0,0.0,2011-12-04 00:00:00,...,clear lens capsule cortex and nucleus,2016-10-22,Parents;Relatives,"416855002.0,160274005.0,160267000.0,160267000.0;;","Diabetes,No history of Diabetes mellitus,No hi...",89,61,60652,M,Unknown
2,11783809.0,335.0,Former smoker,Blue Shield Of Illinois PPO,0.0,0.00,103.0,0.0,0.0,2013-10-16 00:00:00,...,2 + NS (undilated),2016-10-05,Parents,"416855002.0,275134007.0,160357008.0,160267000....","Diabetes,Arthritis,HBP,No history of Glaucoma,...",112,83,60637,F,Unknown
3,6074701.0,1515.0,Never smoker,VOHHA,0.0,0.00,200.0,130.0,80.0,2012-10-01 00:00:00,...,1+ NS,2016-10-10,Parents,"416855002.0,160267000.0,160274005.0,160267000.0;","Diabetes,No history of Glaucoma,No history of ...",146,67,60615,F,Unknown
4,8583237.0,1519.0,Former smoker,Opticare CountyCare,0.0,37.11,118.0,110.0,80.0,2014-07-01 00:00:00,...,vacuoles,2015-12-19,Parents;Siblings,"160274005.0,160267000.0,160274005.0,160267000.0;;","No history of Diabetes mellitus,No history of ...",196,61,60615,F,Unknown


In [29]:
del single_person_all1["Person_Nbr_y"]
single_person_all1=single_person_all1.rename(columns={"Person_Nbr_x":"Person_Nbr"})
single_person_all1.to_csv("single_person_all1.csv")

In [30]:
single_person_all1.to_csv("single_person_all1.csv")